# Except for some extremely minor revisions, all of the credit for this notebook goes to Matteo Esposito!

In [5]:
import numpy as np
import camb

In [6]:
model0 = {
    'ombh2' : 0.022445,
    'omch2' : 0.120567,
    'n_s' : 0.96,
    'A_s' : 2.12723788013000E-09,
    'h' : 0.67,
    'w0' : -1.00,
    'wa' : 0.00,
    'OmK' : 0.00
}          

model0_zs = [
    2.000000,
    1.000000,
    0.570000,
    0.300000,
    0.000000
]

In [7]:
def get_camb_Pk(mlc, omnuh2_in, nu_massive=False, zs=[0], nnu_massive_in=1):
    pars = camb.CAMBparams()
    omch2_in = mlc['omch2']

    print(mlc)
    
    mnu_in = 0
    nnu_massive = 0
    h = mlc["h"]
   
    if nu_massive:
        mnu_in = omnuh2_in * camb.constants.neutrino_mass_fac / \
            (camb.constants.default_nnu / 3.0) ** 0.75 
        #print("The mnu value", mnu_in, "corresponds to the omnuh2 value",
        #    omnuh2_in)
        omch2_in -= omnuh2_in
        nnu_massive = nnu_massive_in

    pars.set_cosmology(
        H0=h*100,
        ombh2=mlc['ombh2'],
        omch2=omch2_in,
        mnu=mnu_in,
        num_massive_neutrinos=nnu_massive,
        omk=mlc['OmK'],
        tau=0.0952,
        neutrino_hierarchy="degenerate"
    )
    
    pars.InitPower.set_params(As=mlc['A_s'], ns=mlc['n_s'],
        r=0, nt=0.0, ntrun=0.0)
    pars.NonLinear = camb.model.NonLinear_none
    pars.WantCls = False
    pars.WantScalars = False
    pars.Want_CMB = False
    pars.DoLensing = False
    pars.YHe = 0.24
    pars.set_accuracy(AccuracyBoost=2)
    pars.Accuracy.AccuracyBoost = 3
    pars.Accuracy.lAccuracyBoost = 3
    pars.Accuracy.AccuratePolarization = False
    pars.Transfer.kmax = 20.0 / h
    
    if mlc['w0'] != -1 or mlc['wa'] != 0:
        pars.set_dark_energy(w=w0, wa=wa, dark_energy_model='ppf') 
        
    # Lukas. I added the "/ h"
    # Lukas. I've verified that for model 0, these redshifts are correct.
    pars.set_matter_power(redshifts=zs, kmax=20.0 / h, nonlinear=False)    
        
    results = camb.get_results(pars)

    sigma12 = results.get_sigmaR(12, hubble_units=False)
    
    # Lukas: I changed var1 and var2 to 8, they were both 2 before
    # Lukas: I changed npoints to 100000, it was 200 before
    kh, z, pkh = results.get_matter_power_spectrum(
        minkh=1e-4/h, maxkh=10.0 / h, npoints = 100000,
        var1=8, var2=8
    ) 
    
    return {'kk': kh*h, 'zz': z, 'Pk': pkh, 's12': sigma12}

In [8]:
omnu_Lukas = 0.002148659574468

LCDM = get_camb_Pk(model0, omnuh2_in=omnu_Lukas, nu_massive=False,
    zs=model0_zs)

mnu_Lukas = 0.1998324308090579 # this is intended to mimic a physical neutrino
# density of 0.002148659574468
mnu_Matteo = 0.3
nuCDM = get_camb_Pk(model0, omnuh2_in=omnu_Lukas, nu_massive=True,
    zs=model0_zs)

{'ombh2': 0.022445, 'omch2': 0.120567, 'n_s': 0.96, 'A_s': 2.12723788013e-09, 'h': 0.67, 'w0': -1.0, 'wa': 0.0, 'OmK': 0.0}
{'ombh2': 0.022445, 'omch2': 0.120567, 'n_s': 0.96, 'A_s': 2.12723788013e-09, 'h': 0.67, 'w0': -1.0, 'wa': 0.0, 'OmK': 0.0}


In [9]:
np.save('LCDM_tiny.npy', LCDM, allow_pickle=True)
np.save('nuCDM_tiny.npy', nuCDM, allow_pickle=True)